In [ ]:
pip install transformers datasets torch

In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, Trainer, TrainingArguments
from datasets import load_dataset

# Load the SQuAD dataset
dataset = load_dataset("squad")

In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, Trainer, TrainingArguments
from datasets import DatasetDict  # or load_dataset if using Hugging Face datasets

# Tokenizer and model for the teacher
teacher_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
teacher_model = AutoModelForQuestionAnswering.from_pretrained("bert-base-uncased")

# Dataset: Ensure it has 'question', 'context', and 'answers' fields
# Example format: {"question": [...], "context": [...], "answers": {"text": [...], "answer_start": [...]}}

# Prepare dataset for tokenization and training
def prepare_train_features(examples):
    tokenized_examples = teacher_tokenizer(
        examples["question"],
        examples["context"],
        truncation=True,
        padding="max_length",
        max_length=384,
        return_offsets_mapping=True,  # Needed to map start/end positions
    )

    # Map the start and end character positions to token positions
    start_positions = []
    end_positions = []

    for i, offsets in enumerate(tokenized_examples["offset_mapping"]):
        answer = examples["answers"][i]
        start_char = answer["answer_start"][0]
        end_char = start_char + len(answer["text"][0])

        # Find the token indices corresponding to the start and end character positions
        start_token = None
        end_token = None
        for idx, (start, end) in enumerate(offsets):
            if start <= start_char < end:
                start_token = idx
            if start < end_char <= end:
                end_token = idx

        # Append token positions, fallback to the beginning of the context if not found
        start_positions.append(start_token if start_token is not None else 0)
        end_positions.append(end_token if end_token is not None else len(offsets) - 1)

    tokenized_examples["start_positions"] = start_positions
    tokenized_examples["end_positions"] = end_positions
    tokenized_examples.pop("offset_mapping")  # Remove offset mappings as they're no longer needed

    return tokenized_examples

# Tokenize the dataset
tokenized_datasets = dataset.map(prepare_train_features, batched=True)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    per_device_train_batch_size=16,
    save_steps=10_000,
    save_total_limit=2,  # Keep only the last 2 checkpoints
    logging_dir="./logs",
    logging_steps=500,
)

# Trainer for the teacher model
trainer = Trainer(
    model=teacher_model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=teacher_tokenizer,
)

# Fine-tune the teacher model
trainer.train()


In [ ]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, Trainer, TrainingArguments
from datasets import DatasetDict  # or load_dataset if using Hugging Face datasets

# Tokenizer and model for the teacher
teacher_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
teacher_model = AutoModelForQuestionAnswering.from_pretrained("bert-base-uncased")

# Prepare dataset for tokenization and training
def prepare_train_features(examples):
    tokenized_examples = teacher_tokenizer(
        examples["question"],
        examples["context"],
        truncation=True,
        padding="max_length",
        max_length=384,
        return_offsets_mapping=True,  # Needed to map start/end positions
    )

    # Map the start and end character positions to token positions
    start_positions = []
    end_positions = []

    for i, offsets in enumerate(tokenized_examples["offset_mapping"]):
        answer = examples["answers"][i]
        start_char = answer["answer_start"][0]
        end_char = start_char + len(answer["text"][0])

        # Find the token indices corresponding to the start and end character positions
        start_token = None
        end_token = None
        for idx, (start, end) in enumerate(offsets):
            if start <= start_char < end:
                start_token = idx
            if start < end_char <= end:
                end_token = idx

        # Append token positions, fallback to the beginning of the context if not found
        start_positions.append(start_token if start_token is not None else 0)
        end_positions.append(end_token if end_token is not None else len(offsets) - 1)

    tokenized_examples["start_positions"] = start_positions
    tokenized_examples["end_positions"] = end_positions
    tokenized_examples.pop("offset_mapping")  # Remove offset mappings as they're no longer needed

    return tokenized_examples

# Assume 'dataset' contains your train and validation splits
# Tokenize the dataset
tokenized_datasets = dataset.map(prepare_train_features, batched=True)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",              # Save directory for checkpoints
    evaluation_strategy="epoch",        # Evaluate at the end of each epoch
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    per_device_train_batch_size=16,
    save_steps=1500,                     # Save checkpoint every 1500 steps
    save_total_limit=3,                  # Keep only the last 3 checkpoints
    logging_dir="./logs",
    logging_steps=500,
)

# Trainer for the teacher model
trainer = Trainer(
    model=teacher_model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=teacher_tokenizer,
)

# Fine-tune the teacher model, resuming from the last checkpoint if available
trainer.train(resume_from_checkpoint=False)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

...

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss


In [ ]:
from transformers import AutoModelForQuestionAnswering, DistilBertTokenizer
import torch
from torch.utils.data import DataLoader

# Student model and tokenizer
student_tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
student_model = AutoModelForQuestionAnswering.from_pretrained("distilbert-base-uncased")

# Tokenize dataset for the student
tokenized_datasets_student = dataset.map(
    lambda examples: student_tokenizer(
        examples["question"], examples["context"], truncation=True, padding="max_length", max_length=384
    ),
    batched=True,
)

# Prepare data loaders
train_loader = DataLoader(tokenized_datasets_student["train"], batch_size=16, shuffle=True)
eval_loader = DataLoader(tokenized_datasets_student["validation"], batch_size=16)

# Loss function for knowledge distillation
def distillation_loss(student_logits, teacher_logits, labels, alpha=0.5, temperature=2.0):
    student_probs = torch.nn.functional.softmax(student_logits / temperature, dim=-1)
    teacher_probs = torch.nn.functional.softmax(teacher_logits / temperature, dim=-1)
    soft_loss = torch.nn.functional.kl_div(student_probs.log(), teacher_probs, reduction="batchmean")
    hard_loss = torch.nn.functional.cross_entropy(student_logits, labels)
    return alpha * soft_loss + (1 - alpha) * hard_loss

# Training loop
optimizer = torch.optim.AdamW(student_model.parameters(), lr=2e-5)

student_model.train()
teacher_model.eval()  # Ensure the teacher model isn't updated

for batch in train_loader:
    input_ids = batch["input_ids"]
    attention_mask = batch["attention_mask"]
    start_positions = batch["start_positions"]
    end_positions = batch["end_positions"]

    with torch.no_grad():
        teacher_outputs = teacher_model(input_ids, attention_mask=attention_mask)

    student_outputs = student_model(input_ids, attention_mask=attention_mask)

    start_loss = distillation_loss(
        student_outputs.start_logits, teacher_outputs.start_logits, start_positions
    )
    end_loss = distillation_loss(
        student_outputs.end_logits, teacher_outputs.end_logits, end_positions
    )
    loss = (start_loss + end_loss) / 2

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


In [ ]:
from transformers import pipeline

# Load fine-tuned student model
qa_pipeline = pipeline("question-answering", model=student_model, tokenizer=student_tokenizer)

# Test a sample question
context = "Knowledge distillation is a technique in machine learning."
question = "What is knowledge distillation?"
result = qa_pipeline(question=question, context=context)
print(result)


In [ ]:

from transformers import AutoModelForQuestionAnswering, AutoTokenizer
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from transformers import default_data_collator

# Load a smaller student model
student_tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
student_model = AutoModelForQuestionAnswering.from_pretrained("distilbert-base-uncased")

# Reuse the same tokenized dataset (re-tokenizing is better for student but skipped for brevity)
train_loader = DataLoader(tokenized_datasets["train"], batch_size=8, shuffle=True, collate_fn=default_data_collator)
val_loader = DataLoader(tokenized_datasets["validation"], batch_size=8, collate_fn=default_data_collator)

# Define optimizer and loss
optimizer = torch.optim.AdamW(student_model.parameters(), lr=3e-5)
ce_loss_fn = nn.CrossEntropyLoss()
kl_loss_fn = nn.KLDivLoss(reduction="batchmean")

student_model.train()
teacher_model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
teacher_model.to(device)
student_model.to(device)

temperature = 2.0
alpha = 0.7
epochs = 3

for epoch in range(epochs):
    total_loss = 0
    for batch in train_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        start_positions = batch["start_positions"].to(device)
        end_positions = batch["end_positions"].to(device)

        # Teacher predictions (no gradient)
        with torch.no_grad():
            teacher_outputs = teacher_model(input_ids=input_ids, attention_mask=attention_mask)
            soft_start = torch.log_softmax(teacher_outputs.start_logits / temperature, dim=-1)
            soft_end = torch.log_softmax(teacher_outputs.end_logits / temperature, dim=-1)

        # Student predictions
        student_outputs = student_model(input_ids=input_ids, attention_mask=attention_mask)
        student_start = student_outputs.start_logits
        student_end = student_outputs.end_logits

        # Compute distillation loss
        loss_start = alpha * kl_loss_fn(torch.log_softmax(student_start / temperature, dim=-1), soft_start) * (temperature ** 2)
        loss_end = alpha * kl_loss_fn(torch.log_softmax(student_end / temperature, dim=-1), soft_end) * (temperature ** 2)

        ce_start = (1 - alpha) * ce_loss_fn(student_start, start_positions)
        ce_end = (1 - alpha) * ce_loss_fn(student_end, end_positions)

        loss = loss_start + loss_end + ce_start + ce_end
        total_loss += loss.item()

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    print(f"Epoch {epoch+1} - Total Loss: {total_loss:.4f}")

# Save student model
student_model.save_pretrained("student_model")
student_tokenizer.save_pretrained("student_model")


In [ ]:

from datasets import load_metric
import numpy as np

# Load SQuAD evaluation metric
metric = load_metric("squad")

# Helper to extract predicted answer from model logits
def get_answer(example, start_logits, end_logits, tokenizer):
    input_ids = example["input_ids"]
    start_index = torch.argmax(start_logits)
    end_index = torch.argmax(end_logits)

    answer_ids = input_ids[start_index : end_index + 1]
    return tokenizer.decode(answer_ids, skip_special_tokens=True)

# Evaluate on a few examples from validation set
student_model.eval()
predictions = []
references = []

for example in val_loader:
    input_ids = example["input_ids"].to(device)
    attention_mask = example["attention_mask"].to(device)
    start_positions = example["start_positions"]
    end_positions = example["end_positions"]
    example_ids = input_ids.cpu().numpy()

    with torch.no_grad():
        outputs = student_model(input_ids=input_ids, attention_mask=attention_mask)

    for i in range(input_ids.size(0)):
        start_logit = outputs.start_logits[i]
        end_logit = outputs.end_logits[i]
        ans = get_answer(example, start_logit, end_logit, student_tokenizer)
        gold_text = student_tokenizer.decode(input_ids[i][start_positions[i]: end_positions[i]+1], skip_special_tokens=True)

        predictions.append({"id": str(i), "prediction_text": ans})
        references.append({"id": str(i), "answers": {"text": [gold_text], "answer_start": [0]}})

# Compute metrics
results = metric.compute(predictions=predictions, references=references)
print(f"F1 Score: {results['f1']:.2f}")
print(f"Exact Match: {results['exact_match']:.2f}")
